# Names: Harrison Fries

## Net IDs: Htf218

# Problem 1

In [68]:
import pandas as pd
import numpy as np
from plotnine import *
%matplotlib inline
from sklearn.neighbors import KNeighborsRegressor as knn
from sklearn.model_selection import train_test_split

In [69]:
df = pd.read_csv("HousingData.csv")
df = df.dropna()
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3,222,18.7,394.12,5.21,28.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499,0.17783,0.0,9.69,0.0,0.585,5.569,73.5,2.3999,6,391,19.2,395.77,15.10,17.5
500,0.22438,0.0,9.69,0.0,0.585,6.027,79.7,2.4982,6,391,19.2,396.90,14.33,16.8
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9


In [70]:
# Creating test and train dataframes by splitting the housing df
train, test = train_test_split(df,
                               test_size=0.2, 
                               random_state=42,
                               shuffle=True)
print("The size of the housing dataset", df.shape[0])
print("The size of the train dataset", train.shape[0])
print("The size of the test dataset",test.shape[0] )

The size of the housing dataset 394
The size of the train dataset 315
The size of the test dataset 79


In [71]:
# general function for fitting testing and training data and scoring each parameter
def test_and_train(columnName: str):
    

    columnScore = columnName + "score"
    scoretrain=[knn(n_neighbors=i).fit(train[[columnName]],
                train['MEDV']).score(train[[columnName]],
                train['MEDV']) for i in list(range(5, 79))]

    trainscore=pd.DataFrame(scoretrain, columns=[columnScore])
    
    trainscore['datsplit']='Training'
    trainscore['neighbors']=list(range(5, 79))

    scoretest=[knn(n_neighbors=i).fit(train[[columnName]],
                train['MEDV']).score(test[[columnName]],
                test['MEDV']) for i in list(range(5, 79))]

    testscore=pd.DataFrame(scoretest, columns=[columnScore])
    
    testscore['datsplit']='Testing'
    testscore['neighbors']=list(range(5, 79))

    score_df = pd.concat([trainscore, testscore], axis=0)

    (ggplot(score_df, aes(x='neighbors', y=columnScore, color='datsplit'))+
    geom_line()+
    scale_color_manual(values=['darkorange', 'steelblue'])+
     theme_bw()
    )

    print(testscore[columnScore].max())
    print(testscore.loc[testscore[columnScore]==testscore[columnScore].max(), 
                  'neighbors'])



In [72]:
# iterate through all columns and run test/train function for each column name
for col in df.columns:
    # only run test/fit on variable columns, not prediction
    if(col != "MEDV"):
        print(col)
        print("Score:")
        test_and_train(col)
        print()
    
    

CRIM
Score:
0.0899974304372847
25    30
Name: neighbors, dtype: int64

ZN
Score:
0.1821375168483852
2    7
Name: neighbors, dtype: int64

INDUS
Score:
0.168892755861085
39    44
Name: neighbors, dtype: int64

CHAS
Score:
-0.0062210639736766105
14    19
Name: neighbors, dtype: int64

NOX
Score:
0.20289382077450124
11    16
Name: neighbors, dtype: int64

RM
Score:
0.49355856382443675
6    11
Name: neighbors, dtype: int64

AGE
Score:
0.047869097113035
38    43
Name: neighbors, dtype: int64

DIS
Score:
0.03228823742219944
0    5
Name: neighbors, dtype: int64

RAD
Score:
0.01720385985625239
8    13
Name: neighbors, dtype: int64

TAX
Score:
0.16613552666279718
2    7
Name: neighbors, dtype: int64

PTRATIO
Score:
0.14659558791086713
26    31
Name: neighbors, dtype: int64

B
Score:
0.09446814346576782
24    29
Name: neighbors, dtype: int64

LSTAT
Score:
0.4565072688915064
18    23
Name: neighbors, dtype: int64



## Final Results
From these test predictions, the average number of rooms per dwelling (RM) is the best predictor, and our optimized hyperparameter is 11 neighbors

# Problem 2

In [73]:
from plotnine import *
from sklearn.ensemble import RandomForestRegressor as rf
from sklearn.ensemble import RandomForestClassifier 
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn import datasets
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

%matplotlib inline

In [74]:
df = pd.read_csv("parkinsons.data")
df.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


We are able to drop the name column as it's not needed for classification.

In [75]:
X = df.drop("status", axis=1)
X = X.drop("name", axis=1)
y = df["status"]

Splitting data into 70% train, 30% test.

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=66)

Creating Random Forrest Model

In [77]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)

RandomForestClassifier()

In [78]:
#predictions
rfc_predict = rfc.predict(X_test)

In [79]:
# Cross Validation Score
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring="roc_auc")

In [80]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')

print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')

print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')

print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[ 5  6]
 [ 2 52]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.71      0.45      0.56        11
           1       0.90      0.96      0.93        54

    accuracy                           0.88        65
   macro avg       0.81      0.71      0.74        65
weighted avg       0.87      0.88      0.87        65



=== All AUC Scores ===
[0.89333333 1.         0.93333333 0.94666667 1.         0.86666667
 0.88333333 0.67857143 0.87142857 0.85      ]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8923333333333334


Our Mean AUC score for our model is .892, but lets see if we can optimize it more.

In [81]:
from sklearn.model_selection import RandomizedSearchCV

In [82]:
#number of trees in random forrest
n_estimators = [int(x) for x in np.linspace(start=200, stop=2000, num=10)]

In [83]:
#features at split
max_features = ['auto', 'sqrt']
#max depth
max_depth = [int(x) for x in np.linspace(100, 500, num=11)]
max_depth.append(None)

In [84]:
random_grid = {
 "n_estimators": n_estimators,
 "max_features": max_features,
 "max_depth": max_depth
}

In [85]:
#random paramater search
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions=random_grid, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

In [86]:
#model fit
rfc_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'max_depth': [100, 140, 180, 220, 260,
                                                      300, 340, 380, 420, 460,
                                                      500, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [87]:
print(rfc_random.best_params_)

{'n_estimators': 200, 'max_features': 'auto', 'max_depth': 220}


Now that our code found that 200 estimators with a max depth of 220 is the "optimal" hyperparameters, let's plug that back into our model.

In [88]:
RandomForestClassifier(n_estimators=200, max_depth=220, max_features='sqrt')
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring='roc_auc')

In [89]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')

print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')

print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')

print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[ 6  5]
 [ 2 52]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.75      0.55      0.63        11
           1       0.91      0.96      0.94        54

    accuracy                           0.89        65
   macro avg       0.83      0.75      0.78        65
weighted avg       0.88      0.89      0.89        65



=== All AUC Scores ===
[0.92       1.         0.89333333 0.97333333 1.         0.86666667
 0.88333333 0.72857143 0.9        0.80714286]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8972380952380952


Based on the score of this RandomForestClassifier, we can see that our hyperparameter changes increased AUC score from .892 to .897.

## Problem 3